In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from simple_ascii_art import img2ascii_arr, img2ascii_arr_v2
from img_processing import preprocess_img, DITHER_MODES

In [2]:
orig_img = Image.open("imgs/irad_grad.bmp").convert("L")

print(orig_img.size)

(512, 512)


In [3]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, scale_factor, contrast, brightness, eq, quantize_colors, dither)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 255, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [9]:
proc_img = interactive_preprocess.result

In [10]:
ascii_arr = img2ascii_arr(proc_img, scale_vertically=True)

In [11]:
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

  ..::;;;;;!;!;!;;;:::..  
..::;;;!!!++++++!!!!;;::..
::;;!!+++oooooooo+++!!;;::
:;;!++ooo444DD4444oo++!!;:
;;!++oo44DDNNNNNDD44o++!!;
;!!+o44DDNNQQQQQNDD4oo+!!;
;!!+o44DNNQ@@@@QNND44o++!;
;!!+oo4DDNNQQQQQNND44o++!;
;!!++oo44DNNNNNNDD44o++!;;
:;;!!+oo444DDD4444oo++!!;:
::;;!!+++ooooooooo++!!;;::
..::;;!!!!++++++!!!!;;::..
  ..:::;;;!!!!!!;;;;::..  


In [12]:
ascii_arr = img2ascii_arr_v2(proc_img)

In [13]:
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

  .::~++++vvvvvvv+++~:::  
.:~+++vcckkkkkkkkcvvv+~~::
:+++vckJJJAAAAAAJJJkccv+~:
++vckJJAAGddddddGGAJJkcv++
++ckJJAGdQQQQQQQQQGGJJkcv+
+ccJJGGQQQQQQQQQQQQGGJJcc+
+ccJJGGQQQQ@@@@QQQQ#GJJ\c+
+ccJJGGQQQQQQQQQQQQGGJJ\c+
+>c\JJG##QQQQQQQQ#GGJJJc++
+++ccJJ2G#######GG2JJ\c>++
::+>cc\JJJ222222JJJ\c>+++:
 ::++>>cc\\\\\\\\\c>+++:: 
   :::+++>>>>>>>++++:::   
